In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [37]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions

def fetch_hourly_rides(hours):
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read()

In [42]:
pd.Timestamp.now(tz="Etc/UTC").floor('h')

Timestamp('2025-02-22 22:00:00+0000', tz='Etc/UTC')

In [43]:
df = fetch_hourly_rides(12)

2025-02-22 17:36:53,518 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-22 17:36:53,529 INFO: Initializing external client
2025-02-22 17:36:53,531 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-22 17:36:54,411 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (7.41s) 


In [44]:
df = df.sort_values(['pickup_hour']).reset_index(drop=True)

In [45]:
df

,pickup_hour,pickup_location_id,rides
0,2025-02-22 05:00:00+00:00,197,0
1,2025-02-22 05:00:00+00:00,41,2
2,2025-02-22 05:00:00+00:00,126,0
3,2025-02-22 05:00:00+00:00,205,1
4,2025-02-22 05:00:00+00:00,253,0
...,...,...,...
2822,2025-02-22 15:00:00+00:00,183,1
2823,2025-02-22 15:00:00+00:00,173,0
2824,2025-02-22 15:00:00+00:00,210,0
2825,2025-02-22 15:00:00+00:00,146,2


In [46]:
df_pred = fetch_predictions(12)

2025-02-22 17:37:17,428 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-22 17:37:17,436 INFO: Initializing external client
2025-02-22 17:37:17,437 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-22 17:37:18,216 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.38s) 


In [47]:
df_pred

,pickup_location_id,predicted_demand,pickup_hour
0,193,2.0,2025-02-22 23:00:00+00:00
1,125,27.0,2025-02-22 23:00:00+00:00
2,72,0.0,2025-02-22 23:00:00+00:00
3,20,0.0,2025-02-22 23:00:00+00:00
4,216,1.0,2025-02-22 23:00:00+00:00
...,...,...,...
252,222,0.0,2025-02-22 23:00:00+00:00
253,122,0.0,2025-02-22 23:00:00+00:00
254,62,0.0,2025-02-22 23:00:00+00:00
255,88,14.0,2025-02-22 23:00:00+00:00


In [34]:
merged_df = pd.merge(df, df_pred, on=['pickup_location_id', 'pickup_hour'])

In [35]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand
